In [1]:
import os
import pickle

if 'notebooks' in os.getcwd():
    os.chdir("../")

In [2]:
## From src/train.py
datafolder = "data"
dataset = "base"
stop = None

import sys
sys.path.append("src")
from build_datasets_utils import *
PADDING = '<PAD>'

#####
train_data = pickle.load(open(os.path.join(datafolder, dataset + "_train_data.pkl"), 'rb'))
valid_data = pickle.load(open(os.path.join(datafolder, dataset + "_valid_data.pkl"), 'rb'))
#labels = pickle.load(open(os.path.join(datafolder, dataset + "_labels.pkl"), 'rb'))
labels = pickle.load(open(os.path.join(datafolder, "labels.pkl"), 'rb'))
print("Training and validation data loaded")

Training and validation data loaded


In [3]:
if stop:
  print("Removing stopwords...")
  train_data = remove_stopwords(train_data)
  valid_data = remove_stopwords(valid_data)
(word_2_idx, vocab) = build_dictionary(train_data, PADDING)
label_map = build_label_map(labels)
print("Labels: {}\n, label map: {}".format(labels, label_map))
train_dataset = FlatData(train_data, word_2_idx, label_map)
valid_dataset = FlatData(valid_data, word_2_idx, label_map)

Deleting UNK
Size of vocabulary: 15477
Top 100 words...
['<PAD>', 'UNK', ':', ',', 'to', 'd', 'you', '?', 'your', 'the', 'and', 'of', 'with', 'discharge', 'dd', 'or', 'a', 'for', 'in', 'please', 'on', 'instructions', 'was', 'at', 'as', 'were', 'have', '', 'diagnosis', 'call', 'is', '-', 'pain', 'no', 'mg', 'daily', 'will', 'condition', 'be', 'dr', 'any', 'if', 'medications', 'this', 'followup', 'up', 'blood', 'by', 'not', 'tablet', 'take', 'that', 'an', 'appointment', 'follow', 'one', 'status', 'weeks', 'ddd', 'from', 'are', 'should', 'care', 'day', 'primary', 'po', 'heart', 'we', 'when', 'left', 'had', 'patient', 'disease', 'it', '(d)', 'sig', 'taking', 'chest', 's/p', 'hospital', 'also', 'stable', 'may', 'medication', 'after', 'than', 'follow-up', 'right', 'failure', 'need', 'artery', 'office', 'which', 'doctor', 'continue', 'home', 'he', 'all', 'cardiac', 'following']

Labels: ['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081']
, label map: {'4019':

In [4]:
print(len(train_dataset))
#train_dataset.get_labels_words(0)
#train_dataset.get_labels_words(len(train_dataset)-1)

27135


In [5]:
print(train_dataset.get_words(0)[0:5])
print(train_dataset.get_dx_index(0))

['discharge', 'diagnosis', ':', 'd', 'hypercholesterolemia']
['41401']


In [6]:
from embedding_utils import *


In [7]:
a = convert_flatdata_to_starspace_format(train_dataset, flag_labeled = True, label_prefix = "__label__")

Finished converting labeled data to StarSpace format.


In [8]:
write_starspace_format(a, os.path.join(datafolder, dataset + "_train_data_starspace.txt"))

Finished writing StarSpace data to file.


In [40]:
starspace_pointer = "test_path"

In [11]:
?map(Bool, 1)

In [13]:
bool(0)

False

In [35]:
from subprocess import run

In [36]:
run([args.starspace, 'train', 
     '-trainFile', os.path.join(args.datafolder, args.dataset + "_train_data_starspace.txt"),
     'model', os.path.join(args.modelfolder, 'labeled_' + args.labeled_embed + "_starspace_embeddings"),
     '-trainMode', '0',
     '-label', '"__label__"'])

AttributeError: 'int' object has no attribute 'stdout'

In [38]:
?subprocess.call

In [46]:
a = map(str, [starspace_pointer, "test_case", ])

In [82]:
ss_paras = ['./Starspace/starspace', 'train', '-trainFile', 'data/base_train_data_starspace.txt', '-model', 'models/labeled_1_starspace_embeddings', '-trainMode', '0', '-label', '__label__', '-dim', '50', '-epoch', '10', '-normalizeText', '0']
#ss_paras = [args.starspace, 'train', '-trainFile', ss_data_path, '-model', ss_model_path, '-trainMode', '0', '-label', '__label__', '-dim', args.embeddim, '-epoch', args.epochs, '-normalizeText', '0']
print(" ".join(ss_paras))

./Starspace/starspace train -trainFile data/base_train_data_starspace.txt -model models/labeled_1_starspace_embeddings -trainMode 0 -label __label__ -dim 50 -epoch 10 -normalizeText 0


In [83]:
ss_output = subprocess.run(ss_paras, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [88]:
print(ss_output.stdout.decode('utf-8'))
print(ss_output.stderr.decode('utf-8'))

Arguments: 
lr: 0.01
dim: 50
epoch: 10
maxTrainTime: 8640000
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 50
thread: 10
minCount: 1
minCountLabel: 1
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 0
fileFormat: fastText
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
Start to initialize starspace model.
Build dict from input file : data/base_train_data_starspace.txt
Loading data from file : data/base_train_data_starspace.txt
Total number of examples loaded : 27135
Training epoch 0: 0.01 0.001

 ---+++                Epoch    0 Train error : 0.03137254 +++--- ☃
Training epoch 1: 0.009 0.001

 ---+++                Epoch    1 Train error : 0.02079385 +++--- ☃
Training epoch 2: 0.008 0.001

 ---+++                Epoch    2 Train error : 0.01829440 +++--- ☃
Training epoch 3: 0.007 0.001

 ---+++                Epoch    3 Train error : 0.01610263 +++--- ☃
Training epoch 4: 0.006 0.001

 ---+++                Epoch    4 Train 

In [98]:
last_output = " ".join(ss_output.stdout.decode("utf-8").split('\n')[-2:])

if "Saving model in tsv format" not in last_output:
    print('Starspace did not complete. PANIC! \nReverting to default initialization.')
    args.init_embed = 0